In [1]:
import pandas as pd
from pathlib import Path
import io, requests

All columns from the kaggle dataset:

`Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type'],
      dtype='object')`

In [67]:
# ---------------------------------------------------------------------
# 1.  Load Kaggle game metadata (date, season, id)
# ---------------------------------------------------------------------
CSV_DIR   = Path("data/csv")
DATA_DIR  = Path("data")

game_df = pd.read_csv(CSV_DIR / "game.csv",
                      usecols=["game_id", "game_date", "season_id", "wl_home", 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'plus_minus_home', 'wl_away',
       'matchup_away', 
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away'])
game_df["date_game"] = pd.to_datetime(game_df["game_date"]).dt.date

In [68]:
# ---------------------------------------------------------------------
# 2.  Load Kaggle line scores (already 1 row per TEAM per GAME)
# ---------------------------------------------------------------------
line_df = pd.read_csv(CSV_DIR / "line_score.csv",
                      usecols=["game_id", "team_abbreviation_home", "team_abbreviation_away", "pts_home"])

# bring in the date + season
kaggle_long = line_df.merge(game_df,
                            on="game_id", how="inner")

In [69]:
# ---------------------------------------------------------------------
# 3.  Load & reshape FiveThirtyEight Elo → long format
# ---------------------------------------------------------------------
elo_538_raw = pd.read_csv(DATA_DIR / "nbaallelo.csv")
elo_538_raw["date_game"] = pd.to_datetime(elo_538_raw["date_game"]).dt.date

elo_team1 = (
    elo_538_raw[["date_game", "team_id", "elo_i", "elo_n", "is_playoffs"]]
      .rename(columns={"team_id": "TEAM_ABBREVIATION",
                       "elo_i": "elo_pre_538",
                       "elo_n": "elo_post_538"})
)
elo_team2 = (
    elo_538_raw[["date_game", "opp_id", "opp_elo_i", "opp_elo_n", "is_playoffs"]]
      .rename(columns={"opp_id": "TEAM_ABBREVIATION",
                       "opp_elo_i": "elo_pre_538",
                       "opp_elo_n": "elo_post_538"})
)

elo_538_long = pd.concat([elo_team1, elo_team2], ignore_index=True)

In [70]:
# ---------------------------------------------------------------------
# 4.  Reconcile historical team codes (minimal starter dict)
# ---------------------------------------------------------------------
alias = {
    "NJN": "BKN", "BRK": "BKN",   # Nets
    "NOH": "NOP", "NOK": "NOP",   # Pelicans
    "CHH": "CHA",                 # Old Hornets
    "SEA": "OKC",                 # Sonics → Thunder
}
kaggle_long["team_abbreviation_home"] = kaggle_long["team_abbreviation_home"].replace(alias)

kaggle_long.rename(columns={"team_abbreviation_home": "TEAM_ABBREVIATION"}, inplace=True)

elo_538_long.rename(columns={"team_id": "TEAM_ABBREVIATION"}, inplace=True)
elo_538_long["TEAM_ABBREVIATION"] = elo_538_long["TEAM_ABBREVIATION"].replace(alias)


In [92]:
# ---------------------------------------------------------------------
# 5.  Merge on date + franchise code
# ---------------------------------------------------------------------

# TO DO: Merge in a way that keeps both teams that played the game
# TO do: The source of the problem is that Kaggle has one row per game,
# and FiveThirtyEight has one row per team per game. Need to find whether Kaggle has both teams listed as separate rows before they were put into long format.
# Problem is that we merged based on date and team code 
merged = kaggle_long.merge(
    elo_538_long,
    on=["date_game", "TEAM_ABBREVIATION"],
    how="inner"
)

In [93]:
merged.columns

Index(['game_id', 'TEAM_ABBREVIATION', 'pts_home', 'team_abbreviation_away',
       'season_id', 'game_date', 'wl_home', 'fgm_home', 'fga_home',
       'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home',
       'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home',
       'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'plus_minus_home', 'matchup_away', 'wl_away', 'fgm_away', 'fga_away',
       'fg_pct_away', 'fg3m_away', 'fg3a_away', 'fg3_pct_away', 'ftm_away',
       'fta_away', 'ft_pct_away', 'oreb_away', 'dreb_away', 'reb_away',
       'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away',
       'plus_minus_away', 'date_game', 'elo_pre_538', 'elo_post_538',
       'is_playoffs'],
      dtype='object')

In [107]:
# Currently, the merged DataFrame has duplicate rows for each game. We need to take each duplicate and swap the columns for the two teams, so that we have a single row per game with both teams' data.

# Taking every other row and swapping the columns for the two teams
copy_1_df = merged[::2]

# Swap 'A' and 'B' column names
swap_mapping = {'TEAM_ABBREVIATION': 'team_abbreviation_away_temp', 'pts_home':'pts_away_temp', 'wl_home':'wl_away_temp',
                'fgm_home':'fgm_away_temp', 'fga_home':'fga_away_temp', 'fg_pct_home':'fg_pct_away_temp', 
                'fg3m_home':'fg3m_away_temp', 'fg3a_home':'fg3a_away_temp', 'fg3_pct_home':'fg3_pct_away_temp',
       'ftm_home':'ftm_away_temp', 'fta_home':'fta_away_temp', 'ft_pct_home':'ft_pct_away_temp', 'oreb_home':'oreb_away_temp', 'dreb_home': 'dreb_away_temp',
       'reb_home':'reb_away_temp', 'ast_home':'ast_away_temp', 'stl_home':'stl_away_temp', 'blk_home':'blk_away_temp',
       'tov_home':'tov_away_temp', 'pf_home':'pf_away_temp', 'plus_minus_home':'plus_minus_away_temp', 'matchup_home':'matchup_away_temp',
       # away names also being swapped
       'team_abbreviation_away':'TEAM_ABBREVIATION_temp', 'pts_away':'pts_home_temp', 'matchup_away':'matchup_home_temp',
       'wl_away':'wl_home_temp', 'fgm_away':'fgm_home_temp', 'fga_away':'fga_home_temp', 'fg_pct_away':'fg_pct_home_temp',
       'fg3m_away':'fg3m_home_temp', 'fg3a_away':'fg3a_home_temp', 
       'fg3_pct_away':'fg3_pct_home_temp', 'ftm_away':'ftm_home_temp', 'fta_away':'fta_home_temp', 'ft_pct_away':'ft_pct_home_temp', 'oreb_away':'oreb_home_temp',
       'dreb_away':'dreb_home_temp', 'reb_away':'reb_home_temp', 'ast_away':'ast_home_temp', 'stl_away':'stl_home_temp', 
       'blk_away':'blk_home_temp', 'tov_away':'tov_home_temp', 'pf_away':'pf_home_temp', 'plus_minus_away':"plus_minus_home_temp"} # Using temporary names
copy_1_df.rename(columns=swap_mapping, inplace=True)

# Now rename the temporary names to their final desired names
final_rename_mapping = {'team_abbreviation_away_temp':'team_abbreviation_away', 'pts_away_temp': 'pts_away', 
                        'wl_away_temp': 'wl_away', 'fgm_away_temp': 'fgm_away', 'fga_away_temp': 'fga_away',
                        'fg_pct_away_temp': 'fg_pct_away', 'fg3m_away_temp': 'fg3m_away', 'fg3a_away_temp': 'fg3a_away',
                        'fg3_pct_away_temp': 'fg3_pct_away', 'ftm_away_temp': 'ftm_away', 'fta_away_temp': 'fta_away',
                        'ft_pct_away_temp': 'ft_pct_away', 'oreb_away_temp': 'oreb_away', 'dreb_away_temp': 'dreb_away',
                        'reb_away_temp': 'reb_away', 'ast_away_temp': 'ast_away', 'stl_away_temp': 'stl_away', 'matchup_away_temp': 'matchup_away',
                        'blk_away_temp': 'blk_away', 'tov_away_temp': 'tov_away', 'pf_away_temp': 'pf_away', 'plus_minus_away_temp':'plus_minus_away',
                        # Now swap the home team columns back to their original names, with the away and home teams swapped
                        "TEAM_ABBREVIATION_temp": "TEAM_ABBREVIATION", "matchup_home_temp":"matchup_home", 'wl_home_temp': "wl_home",
                        "pts_home_temp": "pts_home", "fgm_home_temp": "fgm_home", "fga_home_temp": "fga_home",
                        "fg_pct_home_temp": "fg_pct_home", "fg3m_home_temp": "fg3m_home", "fg3a_home_temp": "fg3a_home",
                        "fg3_pct_home_temp": "fg3_pct_home", "ftm_home_temp": "ftm_home", "fta_home_temp": "fta_home",
                        "ft_pct_home_temp": "ft_pct_home", "oreb_home_temp": "oreb_home", "dreb_home_temp": "dreb_home",
                        "reb_home_temp": "reb_home", "ast_home_temp": "ast_home", "stl_home_temp": "stl_home",
                        "blk_home_temp": "blk_home", "tov_home_temp": "tov_home", "pf_home_temp": "pf_home", "plus_minus_home_temp": "plus_minus_home"
                        }
copy_1_df.rename(columns=final_rename_mapping, inplace=True)

# new_column_order = ['game_id', 'TEAM_ABBREVIATION', 'pts_home', 'team_abbreviation_away',
#        'season_id', 'game_date', 'wl_home', 'fgm_home', 'fga_home',
#        'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home',
#        'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home',
#        'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
#        'plus_minus_home', 'matchup_away', 'wl_away', 'fgm_away', 'fga_away',
#        'fg_pct_away', 'fg3m_away', 'fg3a_away', 'fg3_pct_away', 'ftm_away',
#        'fta_away', 'ft_pct_away', 'oreb_away', 'dreb_away', 'reb_away',
#        'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away',
#        'plus_minus_away', 'date_game', 'elo_pre_538', 'elo_post_538',
#        'is_playoffs']

# copy_1_df = copy_1_df[new_column_order]

/var/folders/xq/z90fxfv513nfkp28682x0phh0000gn/T/ipykernel_64177/467843288.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_1_df.rename(columns=swap_mapping, inplace=True)
/var/folders/xq/z90fxfv513nfkp28682x0phh0000gn/T/ipykernel_64177/467843288.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_1_df.rename(columns=final_rename_mapping, inplace=True)


In [85]:
copy_2_df.columns

Index(['game_id', 'TEAM_ABBREVIATION', 'pts_home', 'team_abbreviation_away',
       'season_id', 'game_date', 'wl_home', 'fgm_home', 'fga_home',
       'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home',
       'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home',
       'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'plus_minus_home', 'matchup_away', 'wl_away', 'fgm_away', 'fga_away',
       'fg_pct_away', 'fg3m_away', 'fg3a_away', 'fg3_pct_away', 'ftm_away',
       'fta_away', 'ft_pct_away', 'oreb_away', 'dreb_away', 'reb_away',
       'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away',
       'plus_minus_away', 'date_game', 'elo_pre_538', 'elo_post_538',
       'is_playoffs'],
      dtype='object')

In [108]:
copy_2_df = merged[1::2]
final_merged = pd.concat([copy_1_df, copy_2_df], axis=0, ignore_index=True)
#final_merged = final_merged.sort_values(by='game_date', ascending=True)


In [112]:
final_merged[final_merged["game_id"] == 41400226]

,game_id,team_abbreviation_away,pts_away,TEAM_ABBREVIATION,season_id,game_date,wl_away,fgm_away,fga_away,fg_pct_away,...,blk_home,tov_home,pf_home,pts_home,plus_minus_home,date_game,elo_pre_538,elo_post_538,is_playoffs,matchup_away
40351,41400226,MEM,95.0,GSW,42014,2015-05-15 00:00:00,L,34.0,91.0,0.374,...,8.0,12.0,24.0,108.0,13,2015-05-15,1618.0546,1608.3455,1,NaN
80717,41400226,GSW,108.0,MEM,42014,2015-05-15 00:00:00,W,41.0,81.0,0.506,...,5.0,3.0,16.0,95.0,-13,2015-05-15,1618.0546,1608.3455,1,GSW @ MEM


In [113]:
# ---------------------------------------------------------------------
# 6.  Save the merged DataFrame to a CSV file
# ---------------------------------------------------------------------
final_merged.to_csv("data/raw/combined_kaggle_538_elo.csv", index=False)